In [ ]:
# How to install sklearn-onnx
# https://github.com/onnx/sklearn-onnx
# I tried "conda instal skl2onnx" but conda couldn't find such package.
# "pip install skl2onnx" successfully installed.

# Tutorial
# http://onnx.ai/sklearn-onnx/introduction.html

import numpy as np
import pandas as pd
import seaborn as sns
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
pd.options.display.float_format = '{:.4}'.format

In [ ]:
# dataset musn't have null values

dataset = pd.read_csv('dataset.csv',header=None)
print(f'Total columns include null: {dataset.isnull().sum().sum()}')

In [ ]:
X = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1].astype(int)

from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

print(y_te)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

classifier.fit(X_tr,y_tr)
y_pred = classifier.predict(X_te)

from sklearn.metrics import confusion_matrix

cn = confusion_matrix(y_te, y_pred)
print(cn)

# Convert sklearn model into ONNX format
initial_type = [('float_input', FloatTensorType([None, len(X.columns)]))]
onx = convert_sklearn(classifier, initial_types=initial_type)
with open("my_classifier.onnx", "wb") as f:
    f.write(onx.SerializeToString())